Цей код створює 2 окремі датафрейми:
- df_duration_ouliers - містить аутлаєри по часу поїздки > 24 годин.
- df - містить сирі дані очищені від аутлаєрів > 24 годин.

В обох датафреймах міститься ще 2 додаткові колонки: started_at_dt і ended_at_dt, що містять час початку і кінця поїздки в типі datetime в таймзомі міста Осло.

In [40]:
import pandas as pd
import datetime
import time
import helpful_functions
import pytz

In [41]:
city_name = 'trondheim'        # можливі варіанти: olso, bergen, trondheim

start_month = 5           # 1 - 12
start_year = 2018         # 2019 - 2022

end_month = 6             # 1 - 12
end_year = 2022           # 2019 - 2022

data_folder = './data'

In [42]:
dates_str_list = helpful_functions.generate_months_in_period(
    start_month=start_month, 
    start_year=start_year, 
    end_month=end_month, 
    end_year=end_year, 
    sep="_")

In [44]:
start = time.time()
df = pd.read_csv(f"{data_folder}/{city_name}/{city_name}_{dates_str_list[0]}.csv")
for date in dates_str_list[1:]:
    try:
        df = pd.concat([df, pd.read_csv(f"{data_folder}/{city_name}/{city_name}_{date}.csv")])
    except:
        pass
end = time.time()
print("Зчитування: ", end - start, "с. \n")
print("Розмір початкової таблиці: ", df.shape)

Зчитування:  6.321700096130371 с. 

Розмір початкової таблиці:  (781422, 13)


In [45]:
strptime_pattern = "%Y-%m-%d %H:%M:%S%z"
timezone = pytz.timezone('Europe/Oslo')

start = time.time()
df['started_at_dt'] = df['started_at'].apply(lambda x: datetime.datetime.strptime(x[:-6].split('.')[0] + x[-6:], strptime_pattern).astimezone(timezone))
df['ended_at_dt'] = df['ended_at'].apply(lambda x: datetime.datetime.strptime(x[:-6].split('.')[0] + x[-6:], strptime_pattern).astimezone(timezone))
end = time.time()
print("Трансформування: ", end - start, "с.")
df.head()

Трансформування:  34.44571495056152 с.


,started_at,ended_at,duration,start_station_id,start_station_name,start_station_description,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_description,end_station_latitude,end_station_longitude,started_at_dt,ended_at_dt
0,2018-05-15 09:07:43.961000+00:00,2018-05-15 09:10:33.267000+00:00,169,28,NaN,,63.430457,10.398101,35,NaN,,63.432504,10.400387,2018-05-15 11:07:43+02:00,2018-05-15 11:10:33+02:00
1,2018-05-15 09:07:51.044000+00:00,2018-05-15 09:10:34.085000+00:00,163,28,NaN,,63.430457,10.398101,35,NaN,,63.432504,10.400387,2018-05-15 11:07:51+02:00,2018-05-15 11:10:34+02:00
2,2018-05-15 09:10:50.177000+00:00,2018-05-15 09:15:33.270000+00:00,283,35,NaN,,63.432504,10.400387,66,NaN,,63.435277,10.405814,2018-05-15 11:10:50+02:00,2018-05-15 11:15:33+02:00
3,2018-05-15 09:11:28.089000+00:00,2018-05-15 09:15:34.746000+00:00,246,35,NaN,,63.432504,10.400387,66,NaN,,63.435277,10.405814,2018-05-15 11:11:28+02:00,2018-05-15 11:15:34+02:00
4,2018-05-15 09:14:21.231000+00:00,2018-05-15 09:15:32.835000+00:00,71,35,NaN,,63.432504,10.400387,35,NaN,,63.432504,10.400387,2018-05-15 11:14:21+02:00,2018-05-15 11:15:32+02:00


In [46]:
df_start_stations = df[['start_station_id',
                        'start_station_name',
                           'start_station_description', 
                           'start_station_latitude', 
                           'start_station_longitude']]

In [47]:
df_start_stations = df_start_stations.drop_duplicates(subset=['start_station_id'],
                                                     keep='last')

In [48]:
df_start_stations.shape

(75, 5)

In [49]:
df_start_stations = df_start_stations.rename(columns={'start_station_id': 'station_id',
                                                     'start_station_name': 'station_name',
                                                     'start_station_description': 'station_description',
                                                     'start_station_latitude': 'station_latitude',
                                                     'start_station_longitude': 'station_longitude'})

In [50]:
df_start_stations.to_csv(f'./data/{city_name}_stations.csv', index=False)

In [51]:
df = df[['started_at_dt', 'ended_at_dt', 'duration', 'start_station_id', 'end_station_id']]

In [52]:
df.shape

(781422, 5)

Створюємо 2 датафрейми: 
- df_duration_ouliers - містить аутлаєри по часу поїздки > 24 годин. Потрібний для подальшого аналізу цих аутлаєрів.
- df - містить сирі дані очищені від аутлаєрів > 24 годин. Для подальшого аналізу.

In [53]:
df_duration_outliers = df[df['duration'] > 24 * 60 * 60]
df = df[df['duration'] <= 24 * 60 * 60]

In [54]:
start = time.time()
df.to_csv(f'./data/{city_name}_clear_rides_data.csv', index=False)
end = time.time()
end - start

17.072633028030396

In [55]:
start = time.time()
df_duration_outliers.to_csv(f'./data/{city_name}_duration_outliers-stolen_bikes.csv', index=False)
end = time.time()
end - start

0.007395029067993164